***
#**EDA**
***


Aca vamos a transformar los datos originales, limpiarlos y prepararlos para el análisis. Identificamos un conjunto de datos "df_games", donde nos muestra las estadísticas de todos los jugadores de cada equipo en todos los partidos que juegan durante el año. Teniendo en cuenta los 82 partidos oficiales que se juega por temporada para clasificar a los playoff, los partidos de post-temporada (playoffs) y los de pre-temporada (partidos no oficiales que juegan para ajustar ideas y tácticas).


## df_NBA

In [ ]:
df_NBA = df_games.copy()
df_NBA.drop_duplicates( subset=['GAME_ID', 'PLAYER_ID'], keep='first', inplace=True) # Eliminamos los duplicados
df_NBA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668339 entries, 0 to 668627
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            668339 non-null  int64  
 1   TEAM_ID            668339 non-null  int64  
 2   TEAM_ABBREVIATION  668339 non-null  object 
 3   TEAM_CITY          668339 non-null  object 
 4   PLAYER_ID          668339 non-null  int64  
 5   PLAYER_NAME        668339 non-null  object 
 6   NICKNAME           53037 non-null   object 
 7   START_POSITION     255644 non-null  object 
 8   COMMENT            109668 non-null  object 
 9   MIN                558670 non-null  object 
 10  FGM                558670 non-null  float64
 11  FGA                558670 non-null  float64
 12  FG_PCT             558670 non-null  float64
 13  FG3M               558670 non-null  float64
 14  FG3A               558670 non-null  float64
 15  FG3_PCT            558670 non-null  float64
 16  FT

In [ ]:
df_NBA = df_NBA.groupby( ['GAME_ID','TEAM_ID']).sum()                # Agrupamos las filas por juego y equipo
df_NBA.reset_index(inplace=True)                                     # Reseteamos el indice
df_NBA.drop(['FG_PCT', 'FG3_PCT', 'FT_PCT'], axis=1, inplace=True)

# Volvemos a calcular los porcentajes de tiros luego de borrar las columnas
df_NBA['FG_PCT'] = df_NBA['FGM']/df_NBA['FGA']
df_NBA['FG3_PCT'] = df_NBA['FG3M']/df_NBA['FG3A']
df_NBA['FT_PCT'] = df_NBA['FTM']/df_NBA['FTA']

df_NBA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53047 entries, 0 to 53046
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   GAME_ID     53047 non-null  int64  
 1   TEAM_ID     53047 non-null  int64  
 2   PLAYER_ID   53047 non-null  int64  
 3   FGM         53047 non-null  float64
 4   FGA         53047 non-null  float64
 5   FG3M        53047 non-null  float64
 6   FG3A        53047 non-null  float64
 7   FTM         53047 non-null  float64
 8   FTA         53047 non-null  float64
 9   OREB        53047 non-null  float64
 10  DREB        53047 non-null  float64
 11  REB         53047 non-null  float64
 12  AST         53047 non-null  float64
 13  STL         53047 non-null  float64
 14  BLK         53047 non-null  float64
 15  TO          53047 non-null  float64
 16  PF          53047 non-null  float64
 17  PTS         53047 non-null  float64
 18  PLUS_MINUS  53047 non-null  float64
 19  FG_PCT      53046 non-nul

In [ ]:
# Fila NaN, sin datos en todas las columnas de estadisticas. Lo que afecta al analisis ya que hay 3 filas con el mismo GAME_ID
# siempre debe tener dos filas con el mismo numero de partido, que indican los dos equipos que juegan el mismo.
display(df_NBA[df_NBA['GAME_ID'] == 10500109])

df_NBA = df_NBA.drop(index=335,axis=0)
df_NBA = df_NBA.reset_index().drop('index',axis=1)

,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,...,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,FG_PCT,FG3_PCT,FT_PCT
334,10500109,1610612742,16233,36.0,73.0,5.0,16.0,26.0,35.0,10.0,...,18.0,8.0,9.0,13.0,19.0,103.0,0.0,0.493151,0.312500,0.742857
335,10500109,1610612756,2564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
336,10500109,1610612765,222934,40.0,87.0,4.0,9.0,13.0,15.0,15.0,...,20.0,7.0,11.0,13.0,29.0,97.0,0.0,0.459770,0.444444,0.866667


In [ ]:
df_NBA.sort_values('GAME_ID', inplace=True)  # Ordenamos los partidos de acuerdo a su numero
df_NBA['VICTORY'] = ""                       # Creamos una columna llamada 'VICTORY'

# Llenamos la columna de 'VICTORY' por SI o por NO, de acuerdo a su resultado
for i in range(0, len(df_NBA)-1, 2):

  if df_NBA['PTS'][i] > df_NBA['PTS'][i+1]:
    df_NBA.loc[i, 'VICTORY'] = 'SI'
    df_NBA.loc[i+1, 'VICTORY'] = 'NO'

  elif df_NBA['PTS'][i] < df_NBA['PTS'][i+1]:
    df_NBA.loc[i, 'VICTORY'] = 'NO'
    df_NBA.loc[i+1, 'VICTORY'] = 'SI'

In [ ]:
# Borramos los partidos que empataron, ya que afecta el analisis y porder predecir si un equipo gano o perdio
df_NBA.drop(df_NBA[df_NBA['VICTORY'] == ""].index, inplace=True)

# Unimos dos data sets (para agregar el año de la temporada al df)
seasons = data_NBA[['GAME_ID','SEASON']]
df_NBA = pd.merge(df_NBA, seasons, how='left', on='GAME_ID')

# Creamos una columna con el nombre completo del equipo
teams['TEAM'] = teams['CITY']+ ' '+ teams['NICKNAME']

# Unimos data sets
df_NBA = pd.merge(df_NBA, teams[['TEAM_ID','TEAM']], how='left', on='TEAM_ID')              # Nombre del equipo, que contiene el data set "teams"
df_NBA = pd.merge(df_NBA, data_NBA[['GAME_ID', 'GAME_DATE_EST']], how='left', on='GAME_ID') # Fecha del partido, que contiene el data set "data_NBA"

df_NBA.head()

,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,...,PF,PTS,PLUS_MINUS,FG_PCT,FG3_PCT,FT_PCT,VICTORY,SEASON,TEAM,GAME_DATE_EST
0,10300001,1610612742,17609,34.0,76.0,2.0,8.0,15.0,30.0,12.0,...,34.0,85.0,0.0,0.447368,0.250000,0.500000,NO,2003.0,Dallas Mavericks,2003-10-05
1,10300001,1610612762,38429,32.0,70.0,1.0,7.0,25.0,34.0,9.0,...,26.0,90.0,0.0,0.457143,0.142857,0.735294,SI,2003.0,Utah Jazz,2003-10-05
2,10300002,1610612749,23082,32.0,75.0,2.0,13.0,28.0,40.0,11.0,...,26.0,94.0,0.0,0.426667,0.153846,0.700000,NO,2003.0,Milwaukee Bucks,2003-10-06
3,10300002,1610612763,32790,40.0,81.0,4.0,15.0,21.0,34.0,14.0,...,35.0,105.0,0.0,0.493827,0.266667,0.617647,SI,2003.0,Memphis Grizzlies,2003-10-06
4,10300003,1610612739,26307,38.0,77.0,4.0,6.0,20.0,29.0,12.0,...,33.0,100.0,0.0,0.493506,0.666667,0.689655,SI,2003.0,Cleveland Cavaliers,2003-10-07


In [ ]:
df_NBA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53216 entries, 0 to 53215
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   GAME_ID        53216 non-null  int64         
 1   TEAM_ID        53216 non-null  int64         
 2   PLAYER_ID      53216 non-null  int64         
 3   FGM            53216 non-null  float64       
 4   FGA            53216 non-null  float64       
 5   FG3M           53216 non-null  float64       
 6   FG3A           53216 non-null  float64       
 7   FTM            53216 non-null  float64       
 8   FTA            53216 non-null  float64       
 9   OREB           53216 non-null  float64       
 10  DREB           53216 non-null  float64       
 11  REB            53216 non-null  float64       
 12  AST            53216 non-null  float64       
 13  STL            53216 non-null  float64       
 14  BLK            53216 non-null  float64       
 15  TO             5321

In [ ]:
# Tabla que indica la conferencia de cada equipo segun su ID de equipo, con el fin de unirla despues a la tabla principal
# El campeonato se divide en dos conferencias (Este y Oeste), donde los equipos primero compiten por ser el campeon de cada conferencia
# y luego juegan la final del campeonato.
conference.head()

,TEAM_ID,CONFERENCE
0,1610612743,West
1,1610612763,West
2,1610612740,West
3,1610612756,West
4,1610612746,West


In [ ]:
conference.info() # 210342 filas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210342 entries, 0 to 210341
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   TEAM_ID     210342 non-null  int64 
 1   CONFERENCE  210342 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.2+ MB


In [ ]:
# Elimino los equipos duplicados
conference.drop_duplicates(subset=['TEAM_ID'], keep='first', inplace=True)
conference.duplicated().sum()

0

In [ ]:
# Muestra solo los 30 equipos de la NBA
conference.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 4289
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   TEAM_ID     30 non-null     int64 
 1   CONFERENCE  30 non-null     object
dtypes: int64(1), object(1)
memory usage: 720.0+ bytes


In [ ]:
# Agregamos la conferencia correspondiente al conjunto de datos
df_NBA = pd.merge(df_NBA, conference, how='left', on='TEAM_ID')
df_NBA.head()

,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,...,PTS,PLUS_MINUS,FG_PCT,FG3_PCT,FT_PCT,VICTORY,SEASON,TEAM,GAME_DATE_EST,CONFERENCE
0,10300001,1610612742,17609,34.0,76.0,2.0,8.0,15.0,30.0,12.0,...,85.0,0.0,0.447368,0.250000,0.500000,NO,2003.0,Dallas Mavericks,2003-10-05,West
1,10300001,1610612762,38429,32.0,70.0,1.0,7.0,25.0,34.0,9.0,...,90.0,0.0,0.457143,0.142857,0.735294,SI,2003.0,Utah Jazz,2003-10-05,West
2,10300002,1610612749,23082,32.0,75.0,2.0,13.0,28.0,40.0,11.0,...,94.0,0.0,0.426667,0.153846,0.700000,NO,2003.0,Milwaukee Bucks,2003-10-06,East
3,10300002,1610612763,32790,40.0,81.0,4.0,15.0,21.0,34.0,14.0,...,105.0,0.0,0.493827,0.266667,0.617647,SI,2003.0,Memphis Grizzlies,2003-10-06,West
4,10300003,1610612739,26307,38.0,77.0,4.0,6.0,20.0,29.0,12.0,...,100.0,0.0,0.493506,0.666667,0.689655,SI,2003.0,Cleveland Cavaliers,2003-10-07,East


In [ ]:
# Contiene el nombre del equipo que jugo de local con el ID de dicho partido
home_team = data_NBA[['GAME_ID', 'HOME_TEAM_ID']]
home_team.head()

,GAME_ID,HOME_TEAM_ID
0,22101005,Miami Heat
1,22101006,Chicago Bulls
2,22101007,San Antonio Spurs
3,22101008,Golden State Warriors
4,22101009,Denver Nuggets


In [ ]:
# Aca creamos una columna llamada 'HOME_TEAM_NUM', donde vamos a poner el equipo local y el visitante (con un 1 y un 0)
# Ya que jugar de local o visita afecfa mucho en el resultado de un partido.
df_NBA = pd.merge(df_NBA, home_team, how='left', on='GAME_ID')
df_NBA['HOME_TEAM_NUM'] = ""

In [ ]:
# 1: Equipo local
# 0: Equipo visitante
for i in range(0, len(df_NBA)):

  if df_NBA['TEAM'][i] == df_NBA['HOME_TEAM_ID'][i]:
    df_NBA.loc[i, 'HOME_TEAM_NUM'] = 1
  else:
    df_NBA.loc[i, 'HOME_TEAM_NUM'] = 0

df_NBA.head(5)

,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,...,FG_PCT,FG3_PCT,FT_PCT,VICTORY,SEASON,TEAM,GAME_DATE_EST,CONFERENCE,HOME_TEAM_ID,HOME_TEAM_NUM
0,10300001,1610612742,17609,34.0,76.0,2.0,8.0,15.0,30.0,12.0,...,0.447368,0.250000,0.500000,NO,2003.0,Dallas Mavericks,2003-10-05,West,Utah Jazz,0
1,10300001,1610612762,38429,32.0,70.0,1.0,7.0,25.0,34.0,9.0,...,0.457143,0.142857,0.735294,SI,2003.0,Utah Jazz,2003-10-05,West,Utah Jazz,1
2,10300002,1610612749,23082,32.0,75.0,2.0,13.0,28.0,40.0,11.0,...,0.426667,0.153846,0.700000,NO,2003.0,Milwaukee Bucks,2003-10-06,East,Memphis Grizzlies,0
3,10300002,1610612763,32790,40.0,81.0,4.0,15.0,21.0,34.0,14.0,...,0.493827,0.266667,0.617647,SI,2003.0,Memphis Grizzlies,2003-10-06,West,Memphis Grizzlies,1
4,10300003,1610612739,26307,38.0,77.0,4.0,6.0,20.0,29.0,12.0,...,0.493506,0.666667,0.689655,SI,2003.0,Cleveland Cavaliers,2003-10-07,East,Detroit Pistons,0


Luego hay que dividir la tabla en los partidos de temporada regular y los de playoff, para tener un mejor análisis y comparación de las dos etapas de competición.

Todas las temporadas tiene un total de 82 partidos por equipo, menos en determinadas temporadas como la 11-12 (donde cada equipo jugo aproximadamente 66 partidos), la temporada 19-20 (que jugaron entre 63-67 partidos cada equipo por motivo del COVID) y la temporada 20-21 (aca jugaron un total de 72, para lograr normalizar la situación, ya que por el COVID la competición estuvo suspendida por un tiempo)

Voy a crear una tabla con el año de la temporada regular, y la fecha de inicio y final de la misma; a si como también las fechas de los playoffs de los distintos años. Debido que en todos los años han cambiado el dia de inicio de la post temporada (playoffs). No hay un dia establecido, la temporada generalmente termina en abril y a los dias empiezan los playoffs. Motivo por el cual debo crear la tabla para un análisis más preciso.
Los datos fueron extraídos de fuentes confiables (Pagina de la NBA), dando mas valor a la tabla y a su posterior análisis.

In [ ]:
# Tabla creada en Excel
season_date.head(5)

,SEASON,START_DATE_SEASON,FINISH_DATE_SEASON,START_DATE_PLAYOFF,FINISH_DATE_PLAYOFF
0,2003,2003-10-28,2004-04-14,2004-04-17,2004-06-15
1,2004,2004-11-02,2005-04-20,2005-04-23,2005-06-23
2,2005,2005-11-01,2006-04-19,2006-04-22,2006-06-20
3,2006,2006-10-31,2007-04-18,2007-04-21,2007-06-14
4,2007,2007-10-30,2008-04-16,2008-04-19,2008-06-17


In [ ]:
df_NBA = pd.merge(df_NBA, season_date, how='left', on='SEASON')
df_NBA.head(5)

,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,...,SEASON,TEAM,GAME_DATE_EST,CONFERENCE,HOME_TEAM_ID,HOME_TEAM_NUM,START_DATE_SEASON,FINISH_DATE_SEASON,START_DATE_PLAYOFF,FINISH_DATE_PLAYOFF
0,10300001,1610612742,17609,34.0,76.0,2.0,8.0,15.0,30.0,12.0,...,2003.0,Dallas Mavericks,2003-10-05,West,Utah Jazz,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15
1,10300001,1610612762,38429,32.0,70.0,1.0,7.0,25.0,34.0,9.0,...,2003.0,Utah Jazz,2003-10-05,West,Utah Jazz,1,2003-10-28,2004-04-14,2004-04-17,2004-06-15
2,10300002,1610612749,23082,32.0,75.0,2.0,13.0,28.0,40.0,11.0,...,2003.0,Milwaukee Bucks,2003-10-06,East,Memphis Grizzlies,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15
3,10300002,1610612763,32790,40.0,81.0,4.0,15.0,21.0,34.0,14.0,...,2003.0,Memphis Grizzlies,2003-10-06,West,Memphis Grizzlies,1,2003-10-28,2004-04-14,2004-04-17,2004-06-15
4,10300003,1610612739,26307,38.0,77.0,4.0,6.0,20.0,29.0,12.0,...,2003.0,Cleveland Cavaliers,2003-10-07,East,Detroit Pistons,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15


In [ ]:
# Ponemos las fechas en el mismo formato para poder hacer una comparación con las mismas,
# y de esa forma dividir los partidos en temporada regular y playoffs.
df_NBA['GAME_DATE_EST'] = pd.to_datetime(df_NBA['GAME_DATE_EST'], format='%m/%d/%Y')

La tabla también tiene partidos de pre-temporada, lo que afecta el análisis, ya que no son partidos oficiales (no valen por los puntos) y generalmente los equipos guardan a sus figuras y prueban diferentes tácticas o estrategias. Por lo cual hay que borrar dichos partidos. Los voy a dividir con la etiqueta 'Borrar'.

In [ ]:
# Creo una columna para que me indique si el partido corresponde a uno de temporada regular, playoff o de pre-temporada.
df_NBA['INSTANCE'] = ""

for i in range(0, len(df_NBA)):

  if df_NBA['START_DATE_SEASON'][i] <= df_NBA['GAME_DATE_EST'][i] <= df_NBA['FINISH_DATE_SEASON'][i]:
    df_NBA.loc[i, 'INSTANCE'] = 'Regular season'
  elif df_NBA['START_DATE_PLAYOFF'][i] <= df_NBA['GAME_DATE_EST'][i] <= df_NBA['FINISH_DATE_PLAYOFF'][i]:
    df_NBA.loc[i, 'INSTANCE'] = 'Playoff'
  else:
    df_NBA.loc[i, 'INSTANCE'] = 'Borrar' # Partidos de pre-temporada

df_NBA.head(1000)

,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,...,TEAM,GAME_DATE_EST,CONFERENCE,HOME_TEAM_ID,HOME_TEAM_NUM,START_DATE_SEASON,FINISH_DATE_SEASON,START_DATE_PLAYOFF,FINISH_DATE_PLAYOFF,INSTANCE
0,10300001,1610612742,17609,34.0,76.0,2.0,8.0,15.0,30.0,12.0,...,Dallas Mavericks,2003-10-05,West,Utah Jazz,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Borrar
1,10300001,1610612762,38429,32.0,70.0,1.0,7.0,25.0,34.0,9.0,...,Utah Jazz,2003-10-05,West,Utah Jazz,1,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Borrar
2,10300002,1610612749,23082,32.0,75.0,2.0,13.0,28.0,40.0,11.0,...,Milwaukee Bucks,2003-10-06,East,Memphis Grizzlies,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Borrar
3,10300002,1610612763,32790,40.0,81.0,4.0,15.0,21.0,34.0,14.0,...,Memphis Grizzlies,2003-10-06,West,Memphis Grizzlies,1,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Borrar
4,10300003,1610612739,26307,38.0,77.0,4.0,6.0,20.0,29.0,12.0,...,Cleveland Cavaliers,2003-10-07,East,Detroit Pistons,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Borrar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10900008,1610612742,521513,31.0,79.0,7.0,18.0,36.0,41.0,10.0,...,Dallas Mavericks,2009-10-05,West,Dallas Mavericks,1,2009-10-27,2010-04-14,2010-04-17,2010-06-17,Borrar
996,10900009,1610612763,2621713,39.0,95.0,3.0,15.0,11.0,24.0,24.0,...,Memphis Grizzlies,2009-10-06,West,Washington Wizards,0,2009-10-27,2010-04-14,2010-04-17,2010-06-17,Borrar
997,10900009,1610612764,1019666,33.0,83.0,6.0,16.0,29.0,43.0,15.0,...,Washington Wizards,2009-10-06,East,Washington Wizards,1,2009-10-27,2010-04-14,2010-04-17,2010-06-17,Borrar
998,10900010,1610612739,2128817,28.0,70.0,3.0,15.0,33.0,41.0,10.0,...,Cleveland Cavaliers,2009-10-06,East,Cleveland Cavaliers,1,2009-10-27,2010-04-14,2010-04-17,2010-06-17,Borrar


In [ ]:
df_NBA['INSTANCE'].value_counts()

Regular season    45562
Borrar             4878
Playoff            3008
Name: INSTANCE, dtype: int64

In [ ]:
# Borramos los partidos de preparacion previos al inicio de la temporada
df_NBA.drop(df_NBA[df_NBA['INSTANCE'] == 'Borrar'].index, inplace=True)

# Cambiamos el tipo a entero, ya que hace referencia a años
df_NBA['SEASON'] = df_NBA['SEASON'].astype(int)
df_NBA['HOME_TEAM_NUM'] = df_NBA['HOME_TEAM_NUM'].astype(int)    # Lo cambiamos a entero para un mejor analisis posteriormente

In [ ]:
df_NBA['VICTORY_NUM'] = ""
df_NBA.reset_index(inplace=True)

for i in range( 0 ,len(df_NBA)):
  if df_NBA['VICTORY'][i] == 'SI':
    df_NBA.loc[ i, 'VICTORY_NUM'] = 1
  else:
    df_NBA.loc[ i, 'VICTORY_NUM'] = 0

df_NBA['VICTORY_NUM'] = df_NBA['VICTORY_NUM'].astype(int)

In [ ]:
df_NBA.head()

,index,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,...,GAME_DATE_EST,CONFERENCE,HOME_TEAM_ID,HOME_TEAM_NUM,START_DATE_SEASON,FINISH_DATE_SEASON,START_DATE_PLAYOFF,FINISH_DATE_PLAYOFF,INSTANCE,VICTORY_NUM
0,2924,11900101,1610612746,13324094,34.0,77.0,16.0,41.0,15.0,19.0,...,2020-07-22,West,Los Angeles Clippers,1,2019-10-22,2020-08-14,2020-08-17,2020-10-11,Regular season,1
1,2925,11900101,1610612753,14656393,32.0,76.0,5.0,28.0,21.0,26.0,...,2020-07-22,East,Los Angeles Clippers,0,2019-10-22,2020-08-14,2020-08-17,2020-10-11,Regular season,0
2,2926,11900102,1610612743,9770101,28.0,63.0,10.0,31.0,23.0,28.0,...,2020-07-22,West,Denver Nuggets,1,2019-10-22,2020-08-14,2020-08-17,2020-10-11,Regular season,1
3,2927,11900102,1610612764,19950731,31.0,85.0,9.0,34.0,11.0,16.0,...,2020-07-22,East,Denver Nuggets,0,2019-10-22,2020-08-14,2020-08-17,2020-10-11,Regular season,0
4,2928,11900103,1610612740,20555620,37.0,79.0,13.0,32.0,12.0,15.0,...,2020-07-22,West,Brooklyn Nets,0,2019-10-22,2020-08-14,2020-08-17,2020-10-11,Regular season,1


In [ ]:
df_NBA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48570 entries, 0 to 48569
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                48570 non-null  int64         
 1   GAME_ID              48570 non-null  int64         
 2   TEAM_ID              48570 non-null  int64         
 3   PLAYER_ID            48570 non-null  int64         
 4   FGM                  48570 non-null  float64       
 5   FGA                  48570 non-null  float64       
 6   FG3M                 48570 non-null  float64       
 7   FG3A                 48570 non-null  float64       
 8   FTM                  48570 non-null  float64       
 9   FTA                  48570 non-null  float64       
 10  OREB                 48570 non-null  float64       
 11  DREB                 48570 non-null  float64       
 12  REB                  48570 non-null  float64       
 13  AST                  48570 non-

In [ ]:
df_NBA.shape

(48570, 36)

## df_regular_season

Creamos una tabla de análisis llamada "df_regular_season", donde contiene verdaderamente tiene todos los partidos de los equipos en temporada regular. Muestra mucha informacíon como el nombre del equipo, todas las estadísticas de juego, el resultado del partido, la temporada, la fecha de juego, conferencia del equipo, si fue local o visitante en el partido. Lo necesario para hacer un interesante análisis en cuanto a la temporada de cada equipo en cada año.

In [ ]:
df_regular_season = df_NBA[df_NBA['INSTANCE'] == 'Regular season']
df_regular_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45562 entries, 0 to 45561
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                45562 non-null  int64         
 1   GAME_ID              45562 non-null  int64         
 2   TEAM_ID              45562 non-null  int64         
 3   PLAYER_ID            45562 non-null  int64         
 4   FGM                  45562 non-null  float64       
 5   FGA                  45562 non-null  float64       
 6   FG3M                 45562 non-null  float64       
 7   FG3A                 45562 non-null  float64       
 8   FTM                  45562 non-null  float64       
 9   FTA                  45562 non-null  float64       
 10  OREB                 45562 non-null  float64       
 11  DREB                 45562 non-null  float64       
 12  REB                  45562 non-null  float64       
 13  AST                  45562 non-

## df_playoffs

Creamos la otra tabla importante de análisis "df_playoffs", donde muestra todos los partidos de post-temporada de cada equipo en cada temporada. Los partidos mas importantes de la NBA, con todos los detalles de los mismos. Mostrando todas las estadísticas de juego, equipo, localia, resultado, conferencia, año de temporada, la instancia de ronda (si es un partido de primera ronda, semifinal de conferencia, final de conferencia y las finales del campeonato) y por supuesto lo mas importante el campeón de la NBA en cada temporada.

In [ ]:
# Creo una columna llamada 'PLAYOFFS', para poner la instancia del partido (ronda) correspondiente como lo explique anteriormente
df_playoffs = df_NBA[df_NBA['INSTANCE'] == 'Playoff']
df_playoffs['PLAYOFFS'] = ""
df_playoffs = df_playoffs.sort_values(['TEAM', 'GAME_ID']) #ordeno para tener todos los partidos por equipo y
                                                           #ver hasta donde llego el mismo en cada temporada (año)

victory_playoff = 0         # Variable que voy a utilizar para contar las victorias del equipo,
                            # si el equipo en el mismo año llega primero a 4 victorias que a 4 derrotas pasa de ronda

for i in range(0, len(df_playoffs)):

  if df_playoffs['SEASON'].iloc[i] == df_playoffs['SEASON'].iloc[i-1] and df_playoffs['TEAM_ID'].iloc[i] == df_playoffs['TEAM_ID'].iloc[i-1]:
   # condicion si sigue el mismo equipo y en el mismo año, quiere decir que el equipo va avanzando en la competicion

    if df_playoffs['VICTORY'].iloc[i] == 'SI':
      victory_playoff += 1                     # contador de victorias

    if victory_playoff <= 4:
      df_playoffs['PLAYOFFS'].iloc[i] = 'First round'
      if victory_playoff == 4 and df_playoffs['VICTORY'].iloc[i] == 'NO':
        df_playoffs['PLAYOFFS'].iloc[i] = 'Semi Conference'

    elif victory_playoff <= 8:
      df_playoffs['PLAYOFFS'].iloc[i] = 'Semi Conference'
      if victory_playoff == 8 and df_playoffs['VICTORY'].iloc[i] == 'NO':
        df_playoffs['PLAYOFFS'].iloc[i] = 'Final Conference'

    elif victory_playoff <= 12:
      df_playoffs['PLAYOFFS'].iloc[i] = 'Final Conference'
      if victory_playoff == 12 and df_playoffs['VICTORY'].iloc[i] == 'NO':
        df_playoffs['PLAYOFFS'].iloc[i] = 'The Finals'

    elif victory_playoff < 16:
      df_playoffs['PLAYOFFS'].iloc[i] = 'The Finals'

    elif victory_playoff == 16:
      df_playoffs['PLAYOFFS'].iloc[i] = 'Champion'
  else:
    victory_playoff = 0
    df_playoffs['PLAYOFFS'].iloc[i] = 'First round'
    if df_playoffs['VICTORY'].iloc[i] == 'SI':
      victory_playoff += 1

df_playoffs['PLAYOFFS'].value_counts()

First round         1568
Semi Conference      822
Final Conference     412
The Finals           188
Champion              18
Name: PLAYOFFS, dtype: int64

In [ ]:
# Ordenamos las filas por ID de partido
df_playoffs = df_playoffs.sort_values(['GAME_ID'])
df_playoffs.head(5)

,index,GAME_ID,TEAM_ID,PLAYER_ID,FGM,FGA,FG3M,FG3A,FTM,FTA,...,CONFERENCE,HOME_TEAM_ID,HOME_TEAM_NUM,START_DATE_SEASON,FINISH_DATE_SEASON,START_DATE_PLAYOFF,FINISH_DATE_PLAYOFF,INSTANCE,VICTORY_NUM,PLAYOFFS
45562,50240,40300101,1610612738,20465,29.0,72.0,8.0,14.0,22.0,29.0,...,East,Indiana Pacers,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Playoff,0,First round
45563,50241,40300101,1610612754,18052,39.0,84.0,5.0,12.0,21.0,27.0,...,East,Indiana Pacers,1,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Playoff,1,First round
45564,50242,40300102,1610612738,20465,27.0,67.0,5.0,18.0,31.0,38.0,...,East,Indiana Pacers,0,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Playoff,0,First round
45565,50243,40300102,1610612754,18052,41.0,92.0,9.0,23.0,12.0,19.0,...,East,Indiana Pacers,1,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Playoff,1,First round
45566,50244,40300103,1610612738,20465,32.0,76.0,3.0,14.0,18.0,23.0,...,East,Boston Celtics,1,2003-10-28,2004-04-14,2004-04-17,2004-06-15,Playoff,0,First round


In [ ]:
df_playoffs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3008 entries, 45562 to 48569
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                3008 non-null   int64         
 1   GAME_ID              3008 non-null   int64         
 2   TEAM_ID              3008 non-null   int64         
 3   PLAYER_ID            3008 non-null   int64         
 4   FGM                  3008 non-null   float64       
 5   FGA                  3008 non-null   float64       
 6   FG3M                 3008 non-null   float64       
 7   FG3A                 3008 non-null   float64       
 8   FTM                  3008 non-null   float64       
 9   FTA                  3008 non-null   float64       
 10  OREB                 3008 non-null   float64       
 11  DREB                 3008 non-null   float64       
 12  REB                  3008 non-null   float64       
 13  AST                  3008 no

## df_ranking_season

In [ ]:
# Creamos un nuevo data drame
df_ranking_season = df_regular_season.copy()

# Agregamos una columna para luego calcular la cantidad de partidos jugados por equipo en cada temporada
df_ranking_season['MATCHES'] = 1

# pasamos de tipo objeto a entero
df_ranking_season['VICTORY_NUM'] = df_ranking_season['VICTORY_NUM'].astype(int)

In [ ]:
# Agrupamos y realizamos los calculos correspondientes en cada columna
df_ranking_season = df_ranking_season.groupby(['TEAM', 'SEASON']).agg({'MATCHES':'sum', 'VICTORY_NUM':'sum',
                                                                'FGM':'mean', 'FGA':'mean',
                                                                'FG3M':'mean', 'FG3A':'mean',
                                                                'FTM':'mean','FTA':'mean',
                                                                'OREB':'mean', 'DREB':'mean',
                                                                'REB':'mean', 'AST':'mean',
                                                                'STL':'mean', 'BLK':'mean',
                                                                'TO':'mean', 'PF':'mean',
                                                                'PTS':'mean', 'FG_PCT':'mean',
                                                                'FG3_PCT':'mean', 'FT_PCT':'mean'})
df_ranking_season.head(100)

MATCHES  VICTORY_NUM        FGM        FGA  \
TEAM                SEASON                                               
Atlanta Hawks       2003         82           28  34.500000  79.621951   
                    2004         82           13  35.878049  81.353659   
                    2005         82           26  35.926829  79.219512   
                    2006         82           30  34.524390  77.707317   
                    2007         82           37  36.280488  79.902439   
...                             ...          ...        ...        ...   
Cleveland Cavaliers 2004         82           42  36.463415  81.548780   
                    2005         82           50  35.463415  78.195122   
                    2006         82           50  36.317073  81.195122   
                    2007         82           45  35.817073  81.670732   
                    2008         82           66  36.853659  78.707317   

                                FG3M       FG3A        FTM        FTA  \
TEAM                SEASON                                              
Atlanta Hawks       2003    5.109756  15.231707  18.707317  24.097561   
                    2004    3.707317  11.865854  17.280488  24.317073   
                    2005    5.170732  14.073171  20.195122  26.914634   
                    2006    4.158537  12.658537  20.451220  26.865854   
                    2007    4.682927  13.146341  20.975610  27.158537   
...                              ...        ...        ...        ...   
Cleveland Cavaliers 2004    3.658537  11.024390  19.926829  26.512195   
                    2005    6.060976  17.865854  20.597561  28.268293   
                    2006    6.024390  17.121951  18.097561  26.012195   
                    2007    6.731707  18.829268  18.012195  25.134146   
                    2008    8.000000  20.365854  18.573171  24.536585   

                                 OREB       DREB        REB        AST  \
TEAM                SEASON                                               
Atlanta Hawks       2003    12.146341  30.573171  42.719512  20.097561   
                    2004    13.414634  28.463415  41.878049  19.658537   
                    2005    13.036585  27.219512  40.256098  19.817073   
                    2006    11.902439  28.195122  40.097561  19.182927   
                    2007    12.292683  29.926829  42.219512  22.000000   
...                               ...        ...        ...        ...   
Cleveland Cavaliers 2004    13.621951  28.682927  42.304878  22.573171   
                    2005    11.695122  30.621951  42.317073  19.024390   
                    2006    12.670732  30.841463  43.512195  20.829268   
                    2007    13.317073  31.256098  44.573171  20.000000   
                    2008    10.804878  31.390244  42.195122  20.280488   

                                 STL       BLK         TO         PF  \
TEAM                SEASON                                             
Atlanta Hawks       2003    7.646341  4.975610  15.585366  22.268293   
                    2004    7.646341  4.195122  15.304878  24.500000   
                    2005    7.158537  4.804878  15.060976  25.048780   
                    2006    7.426829  5.439024  15.195122  24.024390   
                    2007    7.317073  5.463415  14.268293  20.402439   
...                              ...       ...        ...        ...   
Cleveland Cavaliers 2004    7.975610  5.621951  13.060976  22.560976   
                    2005    6.914634  4.780488  13.402439  21.134146   
                    2006    7.621951  4.304878  13.743902  21.719512   
                    2007    7.060976  5.207317  13.182927  20.878049   
                    2008    7.231707  5.304878  11.975610  20.280488   

                                   PTS    FG_PCT   FG3_PCT    FT_PCT  
TEAM                SEASON                                            
Atlanta Hawks       2003     92.817073  0.433656  0.319981  0.771988  
  

In [ ]:
# Reseteamos el indice
df_ranking_season.reset_index(inplace=True)

In [ ]:
# Creamos una columna que nos indique el porcentaje de vitoria que tuvo cada equipo en cada temporada en la NBA,
# ya que los ocho mejores de cada conferencia entran a los Playoffs.
df_ranking_season['WIN_PERCENTAGE'] = df_ranking_season['VICTORY_NUM'] / df_ranking_season['MATCHES']

In [ ]:
teams = df_regular_season[['TEAM', 'CONFERENCE']].drop_duplicates(subset=['TEAM'], keep='first')
teams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 2477
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   TEAM        30 non-null     object
 1   CONFERENCE  30 non-null     object
dtypes: object(2)
memory usage: 720.0+ bytes


In [ ]:
df_ranking_season = pd.merge(df_ranking_season, teams, how='left', on='TEAM')
df_ranking_season = df_ranking_season.sort_values(['SEASON', 'CONFERENCE', 'WIN_PERCENTAGE'], ascending=False)

df_ranking_season.head()

,TEAM,SEASON,MATCHES,VICTORY_NUM,FGM,FGA,FG3M,FG3A,FTM,FTA,...,STL,BLK,TO,PF,PTS,FG_PCT,FG3_PCT,FT_PCT,WIN_PERCENTAGE,CONFERENCE
454,Phoenix Suns,2021,67,53,43.044776,89.477612,11.522388,31.791045,16.029851,20.149254,...,8.552239,4.432836,12.432836,19.343284,113.641791,0.481896,0.362159,0.794939,0.791045,West
188,Golden State Warriors,2021,68,46,40.705882,86.764706,14.514706,39.955882,15.441176,20.250000,...,9.220588,4.735294,14.367647,20.647059,111.367647,0.469460,0.363562,0.764358,0.676471,West
283,Memphis Grizzlies,2021,68,46,43.102941,93.911765,11.029412,32.397059,17.014706,23.294118,...,9.867647,6.485294,12.485294,19.632353,114.250000,0.459650,0.341672,0.735795,0.676471,West
549,Utah Jazz,2021,67,42,40.716418,86.104478,14.671642,40.388060,17.761194,22.910448,...,7.104478,4.895522,13.343284,19.044776,113.865672,0.473559,0.362949,0.777855,0.626866,West
131,Dallas Mavericks,2021,67,41,39.074627,85.791045,12.731343,37.134328,15.895522,20.820896,...,6.985075,4.149254,11.701493,19.895522,106.776119,0.456180,0.343651,0.768883,0.611940,West


Tabla de análisis 'df_ranking_season', donde nos muestra el porcentaje de victorias que tuvo cada equipo en cada temporada regular (que es muy importante para definir los puestos y cruces de playoff). También muestra el promedio de todas las estadísticas de juego de cada equipo, año y la conferencia correspondiente.

In [ ]:
df_ranking_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569 entries, 454 to 398
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   TEAM            569 non-null    object 
 1   SEASON          569 non-null    int64  
 2   MATCHES         569 non-null    int64  
 3   VICTORY_NUM     569 non-null    int64  
 4   FGM             569 non-null    float64
 5   FGA             569 non-null    float64
 6   FG3M            569 non-null    float64
 7   FG3A            569 non-null    float64
 8   FTM             569 non-null    float64
 9   FTA             569 non-null    float64
 10  OREB            569 non-null    float64
 11  DREB            569 non-null    float64
 12  REB             569 non-null    float64
 13  AST             569 non-null    float64
 14  STL             569 non-null    float64
 15  BLK             569 non-null    float64
 16  TO              569 non-null    float64
 17  PF              569 non-null    f

In [ ]:
df_ranking_season.shape

(569, 24)

## df_players

En esta tabla tendremos todos los datos y estadísticas importantes a nivel individual de todos los jugadores de la competición (NBA).

In [ ]:
df_players = df_games.copy()
df_players.drop_duplicates( subset=['GAME_ID', 'PLAYER_ID'], keep='first', inplace=True) # Eliminamos los duplicados
df_players.head(5)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0


In [ ]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668339 entries, 0 to 668627
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            668339 non-null  int64  
 1   TEAM_ID            668339 non-null  int64  
 2   TEAM_ABBREVIATION  668339 non-null  object 
 3   TEAM_CITY          668339 non-null  object 
 4   PLAYER_ID          668339 non-null  int64  
 5   PLAYER_NAME        668339 non-null  object 
 6   NICKNAME           53037 non-null   object 
 7   START_POSITION     255644 non-null  object 
 8   COMMENT            109668 non-null  object 
 9   MIN                558670 non-null  object 
 10  FGM                558670 non-null  float64
 11  FGA                558670 non-null  float64
 12  FG_PCT             558670 non-null  float64
 13  FG3M               558670 non-null  float64
 14  FG3A               558670 non-null  float64
 15  FG3_PCT            558670 non-null  float64
 16  FT

In [ ]:
# Agregamos el año de temporada a la tabla:
df_players = pd.merge(df_players, seasons, how='left', on='GAME_ID')

# Agregamos el nombre de los equipos a la tabla:
teams = pd.read_csv('teams.csv')
teams['TEAM'] = teams['CITY']+ ' '+ teams['NICKNAME']   # Nombre compreto de los equipos
df_players = pd.merge(df_players, teams[['TEAM_ID','TEAM']], how='left', on='TEAM_ID')

# Agregamos el numero de victory (si gano o perdio el partido), para calcular cuantos partidos gano cada jugador:
num_victory = df_NBA[['GAME_ID', 'TEAM_ID', 'VICTORY_NUM']]
df_players = pd.merge(df_players, num_victory, how='left', on=['GAME_ID', 'TEAM_ID'])

In [ ]:
# Agregamos una columna que nos indique quienes salieron campeones, para un análisis individual del jugador:
df_playoffs['CHAMPION_NUM'] = ""
df_playoffs['CHAMPION_NUM'] = df_playoffs['PLAYOFFS'].apply(lambda x: 1 if x == 'Champion' else 0)

num_champion = df_playoffs[['GAME_ID', 'TEAM_ID', 'CHAMPION_NUM']]
df_players = pd.merge(df_players, num_champion, how='left', on=['GAME_ID', 'TEAM_ID'])

df_players.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,STL,BLK,TO,PF,PTS,PLUS_MINUS,SEASON,TEAM,VICTORY_NUM,CHAMPION_NUM
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,...,1.0,0.0,2.0,5.0,2.0,-2.0,NaN,San Antonio Spurs,NaN,NaN
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,...,1.0,0.0,2.0,1.0,23.0,-14.0,NaN,San Antonio Spurs,NaN,NaN
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,...,1.0,0.0,2.0,4.0,13.0,-4.0,NaN,San Antonio Spurs,NaN,NaN
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,...,3.0,0.0,2.0,1.0,10.0,-18.0,NaN,San Antonio Spurs,NaN,NaN
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,...,0.0,0.0,2.0,2.0,19.0,0.0,NaN,San Antonio Spurs,NaN,NaN


In [ ]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 680489 entries, 0 to 680488
Data columns (total 33 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            680489 non-null  int64  
 1   TEAM_ID            680489 non-null  int64  
 2   TEAM_ABBREVIATION  680489 non-null  object 
 3   TEAM_CITY          680489 non-null  object 
 4   PLAYER_ID          680489 non-null  int64  
 5   PLAYER_NAME        680489 non-null  object 
 6   NICKNAME           53037 non-null   object 
 7   START_POSITION     259994 non-null  object 
 8   COMMENT            112428 non-null  object 
 9   MIN                568060 non-null  object 
 10  FGM                568060 non-null  float64
 11  FGA                568060 non-null  float64
 12  FG_PCT             568060 non-null  float64
 13  FG3M               568060 non-null  float64
 14  FG3A               568060 non-null  float64
 15  FG3_PCT            568060 non-null  float64
 16  FT

In [ ]:
df_players['SEASON'].isnull().sum()  # Datos nulos

22675

In [ ]:
df_players = df_players.dropna(subset=['SEASON'])
df_players['SEASON'] = df_players['SEASON'].astype(int)
df_players['MATCHES'] = 1

In [ ]:
df_players['PLAYER_NAME'].nunique()   # Cantidad de jugadores en la NBA

2549

In [ ]:
df_players = df_players.groupby(['PLAYER_NAME', 'SEASON']).agg({'MATCHES':'sum',
                                                                'FGM':'sum', 'FGA':'sum',
                                                                'FG3M':'sum', 'FG3A':'sum',
                                                                'FTM':'sum','FTA':'sum',
                                                                'OREB':'sum', 'DREB':'sum',
                                                                'REB':'sum', 'AST':'sum',
                                                                'STL':'sum', 'BLK':'sum',
                                                                'TO':'sum', 'PF':'sum',
                                                                'PTS':'sum', 'FG_PCT':'sum',
                                                                'FG3_PCT':'sum', 'FT_PCT':'sum',
                                                                'VICTORY_NUM':'sum', 'CHAMPION_NUM':'sum'})
df_players.reset_index(inplace=True)

df_players.head(5)

,PLAYER_NAME,SEASON,MATCHES,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,...,STL,BLK,TO,PF,PTS,FG_PCT,FG3_PCT,FT_PCT,VICTORY_NUM,CHAMPION_NUM
0,A.J. Guyton,2003,1,0.0,4.0,0.0,2.0,4.0,4.0,1.0,...,1.0,0.0,0.0,2.0,4.0,0.000,0.000,1.000,0.0,0.0
1,A.J. Lawson,2021,2,2.0,3.0,2.0,2.0,2.0,2.0,0.0,...,0.0,0.0,1.0,0.0,8.0,0.667,1.000,1.000,0.0,0.0
2,AJ Hammons,2016,52,34.0,76.0,5.0,12.0,9.0,20.0,14.0,...,1.0,16.0,14.0,34.0,82.0,8.924,3.833,3.750,14.0,0.0
3,AJ Hammons,2017,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0
4,AJ Price,2009,83,166.0,403.0,67.0,195.0,62.0,81.0,14.0,...,37.0,3.0,65.0,65.0,461.0,24.278,18.813,25.317,32.0,0.0


In [ ]:
# Agregamos una columna para calcular la cantidad de temporadas que disputo en la NBA cada jugador:
df_players['SEASON_NUM'] = 1
df_players = df_players.groupby('PLAYER_NAME').sum()
df_players.reset_index(inplace=True)
df_players.drop(['SEASON'], axis=1, inplace=True)

In [ ]:
# Calculo el promedio de todas las estadísticas de juego teniendo en cuanta la cantidad de partidos jugados:
columns_divide = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM','FTA', 'OREB', 'DREB', 'REB',
                  'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'FG_PCT', 'FG3_PCT', 'FT_PCT']

df_players[columns_divide] = df_players[columns_divide].div(df_players['MATCHES'], axis=0)

Para completar y darle mas valor a la tabla con informacíon relevante, cree una tabla de excel con ganadores de premios como el prestigioso MVP de temporada regular, a si como también los mejores quintetos de cada año (2003-2021). Estas elecciones las da la propia NBA como reconocimiento a los mejores de la temporada, por un lado elige los tres mejores quintetos (mejor quinteto, segundo mejor quinteto y tercer quinteto) son 15 jugadores en total; y por otro lado los dos mejores quintetos pero de la parte defensiva, en este caso serian 10 jugadores premiados.

In [ ]:
# Tabla de los premios:
premios.head()

,PLAYER_NAME,BEST_15,BEST_10_DEFENCE,MVP_SEASON
0,Al Horford,1.0,1.0,NaN
1,Al Jefferson,1.0,NaN,NaN
2,Allen Iverson,2.0,NaN,NaN
3,Amare Stoudemire,5.0,NaN,NaN
4,Anderson Varejao,NaN,1.0,NaN


In [ ]:
# Agregamos las columnas de los premios:
df_players = pd.merge(df_players, premios, how='left', on=['PLAYER_NAME'])

In [ ]:
df_players.isnull().sum()

PLAYER_NAME           0
MATCHES               0
FGM                   0
FGA                   0
FG3M                  0
FG3A                  0
FTM                   0
FTA                   0
OREB                  0
DREB                  0
REB                   0
AST                   0
STL                   0
BLK                   0
TO                    0
PF                    0
PTS                   0
FG_PCT                0
FG3_PCT               0
FT_PCT                0
VICTORY_NUM           0
CHAMPION_NUM          0
SEASON_NUM            0
BEST_15            2468
BEST_10_DEFENCE    2489
MVP_SEASON         2538
dtype: int64

In [ ]:
# Llenado de null por 0 (cero) en las nuevas columnas:
df_players.fillna(0, inplace=True)

In [ ]:
df_players.isnull().sum()

PLAYER_NAME        0
MATCHES            0
FGM                0
FGA                0
FG3M               0
FG3A               0
FTM                0
FTA                0
OREB               0
DREB               0
REB                0
AST                0
STL                0
BLK                0
TO                 0
PF                 0
PTS                0
FG_PCT             0
FG3_PCT            0
FT_PCT             0
VICTORY_NUM        0
CHAMPION_NUM       0
SEASON_NUM         0
BEST_15            0
BEST_10_DEFENCE    0
MVP_SEASON         0
dtype: int64

In [ ]:
df_players.head()

,PLAYER_NAME,MATCHES,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,...,PTS,FG_PCT,FG3_PCT,FT_PCT,VICTORY_NUM,CHAMPION_NUM,SEASON_NUM,BEST_15,BEST_10_DEFENCE,MVP_SEASON
0,A.J. Guyton,1,0.000000,4.000000,0.000000,2.000000,4.000000,4.000000,1.000000,1.000000,...,4.000000,0.000000,0.000000,1.000000,0.0,0.0,1,0.0,0.0,0.0
1,A.J. Lawson,2,1.000000,1.500000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,...,4.000000,0.333500,0.500000,0.500000,0.0,0.0,1,0.0,0.0,0.0
2,AJ Hammons,58,0.586207,1.310345,0.086207,0.206897,0.155172,0.344828,0.241379,0.793103,...,1.413793,0.153862,0.066086,0.064655,14.0,0.0,2,0.0,0.0,0.0
3,AJ Price,415,1.578313,4.062651,0.643373,1.939759,0.614458,0.826506,0.200000,0.848193,...,4.414458,0.249624,0.187631,0.231696,178.0,0.0,6,0.0,0.0,0.0
4,Aamir Simms,4,0.250000,0.500000,0.000000,0.000000,0.500000,0.500000,0.750000,0.500000,...,1.000000,0.250000,0.000000,0.250000,0.0,0.0,1,0.0,0.0,0.0


In [ ]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2549 entries, 0 to 2548
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PLAYER_NAME      2549 non-null   object 
 1   MATCHES          2549 non-null   int64  
 2   FGM              2549 non-null   float64
 3   FGA              2549 non-null   float64
 4   FG3M             2549 non-null   float64
 5   FG3A             2549 non-null   float64
 6   FTM              2549 non-null   float64
 7   FTA              2549 non-null   float64
 8   OREB             2549 non-null   float64
 9   DREB             2549 non-null   float64
 10  REB              2549 non-null   float64
 11  AST              2549 non-null   float64
 12  STL              2549 non-null   float64
 13  BLK              2549 non-null   float64
 14  TO               2549 non-null   float64
 15  PF               2549 non-null   float64
 16  PTS              2549 non-null   float64
 17  FG_PCT        

In [ ]:
df_players.shape

(2549, 26)